In [1]:
import os
import autogen
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_community.document_loaders.image import UnstructuredImageLoader
from autogen import register_function
from typing import Annotated, Literal
from datetime import date

flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.


In [2]:
# def parse_files_in_directory(directory_path):
    
#     parsed_data = parse_pdfs(directory_path)

#     # for filename in os.listdir(directory_path):
#         # file_path = os.path.join(directory_path, filename)
        
#         # if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp')):
#         #     loader = UnstructuredImageLoader(file_path)
#         #     documents = loader.load()
#         #     parsed_text = ' '.join([doc.page_content for doc in documents])

#         # else:
#         #     continue

#         # parsed_data[filename] = parsed_text

#     return parsed_data


def parse_pdfs(directory:str ="../data/") -> dict:
# def parse_pdfs(dir_path="../data/"):
    parsed_data = {}
    loader = PyPDFDirectoryLoader(directory, extract_images=True)
    documents = loader.load()
    for page in documents:
        if parsed_data.get(page.metadata["source"]) is None:
            parsed_data[page.metadata["source"]] = [page.page_content]
        else:
            parsed_data[page.metadata["source"]].append(page.page_content)
    return parsed_data


In [ ]:
api_key = os.environ.get("GROQ_API_KEY")

config_list = [
  {
    "model": "llama-3.2-90b-text-preview",
    "base_url": "https://api.groq.com/openai/v1/chat/completions",
    "api_type": "groq",
    "api_key": api_key,
    "api_rate_limit": 100
  }
]

# llm_config = config_list[0]
llm_config={"config_list": config_list}

In [4]:
# user_proxy = autogen.ConversableAgent(
#     name="Admin",
#     system_message="Give the task, and send instructions to doc_classifier to classify the documents in the data folder",
#     code_execution_config=False,
#     llm_config=llm_config,
#     human_input_mode="ALWAYS",
# )

user_proxy = autogen.ConversableAgent(
    name="User",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    human_input_mode="NEVER",
    # max_consecutive_auto_reply=10,
    # code_execution_config={
    #     "work_dir": "code",
    #     "use_docker": False
    # },
    llm_config=False,
)

In [ ]:
doc_parse_caller = autogen.ConversableAgent(
    name="Doc Parse Caller",
    llm_config=llm_config,
    description="A caller for parsing documents",
    system_message="You are a helpful AI assistant. Use the provided python function parse_pdfs(directory), to parse the documents at the directory '../data/'.",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
)

doc_parser = autogen.ConversableAgent(
    name="Doc Parser",
    llm_config=llm_config,
    system_message="You are an executer that parses documents. If you receive a python call instruction, then you execute it, or else ignore the message."
                   "Reply TERMINATE afer you execute the python function",
)

prompt = """
For each document provided, analyze its content and categorize it into one of the following categories: ['Passport', 'Residence Permit', 'Transaction', 'Health Insurance', 'Enrollment Letter', 'Other']. Your response should strictly follow this dictionary structured format:

{
  "category_name": "<summarized_content>",
}

Always include each key exactly as shown. The category_name field should contain the best matching category from the predefined options, and the summarized_content value should include a summary of the parsed document and keep all the essential information only. 
Adhere precisely to this format of a single dictionary of only string keys and values. DO NOT include file names.
"""
# prompt = """
# For each document provided, analyze its content and categorize it into one of the following categories: ['Passport', 'Residence Permit', 'Transaction', 'Health Insurance', 'Other']. Your response should strictly follow this dictionary structured format:

# {
#   "category_name": "<full_parsed_document_string>",
# }

# Always include each key exactly as shown. The category_name field should contain the best matching category from the predefined options, and the full_parsed_document_string value should include the full parsed document text. Adhere precisely to this format, without adding or omitting any fields.
# """

doc_classifier = autogen.ConversableAgent(
    name="Doc Classifier",
    llm_config=llm_config,
    description="A classifier that classifies the text that has been parsed",
    system_message=prompt
)




# doc_classifier = autogen.ConversableAgent(
#     name="Doc Classifier",
#     llm_config=llm_config,
#     description="A classifier that classifies the text that has been parsed",
#     system_message="For each document provided, categorize them from one of these options ['Passport', 'Residence Permit', 'Transaction', 'Health Insurance', 'Other'] "
#                    "and reply with a dictionary in this format {category_name: full_parsed_document}, where full_parsed_document is the whole document that was parsed"
# )

chief_verifier = autogen.ConversableAgent(
    name="Chief Verifier",
    llm_config=llm_config,
    description="The Chief verifier",
    system_message="You are the chief verifier."
                   "You collect the information from the Doc Classifier and pass the information to your team."
)

# health_insurance_verifier = autogen.ConversableAgent(
#     name="Health Insurance Verifier",
#     llm_config=llm_config,
#     description="A verifier that checks if health insurance is still valid",
#     system_message="You are a helpful AI assistant."
#                    "If there exists a Health Insurance key in the dictionary received, then check its corresponding parsed document to see if the validity of the insurance is beyond today."
#                 #    "Reply TERMINATE when the task is done.",
# )

rp_prompt = f'''
Your role is a Residence Permit Verifier. Your task is to check the contents of a dictionary/dictionaries that contains various document types as keys and their content as values.
If a key named "Residence Permit" is present in the dictionary, examine its content to verify whether the residence permit is valid beyond today’s date which is {date.today()}. 
Provide a precise response indicating whether the residence permit is valid or expired, based on the expiration information in the document. DO NOT use any code to verify the content.
'''

passport_prompt = f'''
Your role is a Passport Verifier. Your task is to check the contents of a dictionary/dictionaries that contains various document types as keys and their content as values.
If a key named "Passport" is present in the dictionary, examine its content to verify whether the document is valid beyond today’s date which is {date.today()}. 
Provide a precise response indicating whether the document is valid or expired, based on the expiration information in the summary content. DO NOT use any code to verify the content.
'''

rp_verifier = autogen.ConversableAgent(
    name="Residence Permit Verifier",
    llm_config=llm_config,
    description="A verifier that checks if residence permit is still valid",
    system_message=rp_prompt
)

passport_verifier = autogen.ConversableAgent(
    name="Passport Verifier",
    llm_config=llm_config,
    description="A verifier that checks passport validity",
    system_message=passport_prompt
    # system_message="You are a helpful AI assistant."
    #                "If there exists a passport document, then check if the expiry date is more than a year from today"
                #    "Reply TERMINATE when the task is done.",
)


edu_cert_prompt = f'''
Your role is a Education Certificate Verifier. Your task is to check the contents of a dictionary/dictionaries that contains various document types as keys and their content as values.
If a key named "Enrollment Letter" is present in the dictionary, examine its content to verify whether the document is in accordance with the guidelines mentioned in the '../data/template/Template_enrollement.pdf' document. 
Provide a precise response indicating whether the document is valid or not, based on the information in the summary content. DO NOT use any code to verify the content.
'''

education_cert_verifier = autogen.ConversableAgent(
    name="Education Certificate Verifier",
    llm_config=llm_config,
    description="A verifier that checks the validity of the education certificate",
    system_message=edu_cert_prompt
)

summary_verifier = autogen.ConversableAgent(
    name="Verification Summarizer",
    llm_config=llm_config,
    description="Summarizer for all verifications provided",
    system_message="You are the summarizer for all verifications provided. Aggregrate the decisions by the verifications provided and provide a detailed remark for each of them",
)



In [6]:
register_function(
    parse_pdfs,
    caller=doc_parse_caller,  
    executor=doc_parser,  
    name="parse_pdfs",  
    description="A simple document parser",  # A description of the tool.
)

# # Register the tool signature with the assistant agent.
# doc_parse_caller.register_for_llm(name="parse_pdfs", description="A simple pdf parser")(parse_pdfs)

# # Register the tool function with the user proxy agent.
# doc_parser.register_for_execution(name="parse_pdfs")(parse_pdfs)

In [7]:
groupchat = autogen.GroupChat(
    # agents=[user_proxy, doc_parse_caller, doc_parser, doc_classifier, chief_verifier, passport_verifier, rp_verifier],
    agents=[user_proxy, doc_parse_caller, doc_parser, doc_classifier, chief_verifier],
    messages=[],
    max_round=5,
    allowed_or_disallowed_speaker_transitions={
        user_proxy: [doc_parse_caller],
        doc_parse_caller: [doc_parser],
        doc_parser: [doc_classifier],
        doc_classifier: [chief_verifier],
        # chief_verifier: [passport_verifier, rp_verifier],
        chief_verifier: [user_proxy],
    },
    speaker_transitions_type="allowed",
)

manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

In [8]:
doc_parse_caller.llm_config["tools"]

[{'type': 'function',
  'function': {'description': 'A simple document parser',
   'name': 'parse_pdfs',
   'parameters': {'type': 'object',
    'properties': {'directory': {'type': 'string',
      'default': '../data/',
      'description': 'directory'}},
    'required': []}}}]

In [9]:
groupchat2 = autogen.GroupChat(
    # agents=[user_proxy, doc_parse_caller, doc_parser, doc_classifier, chief_verifier, passport_verifier, rp_verifier],
    agents=[chief_verifier, doc_parse_caller, doc_parser, education_cert_verifier],
    messages=[],
    max_round=5,
    allowed_or_disallowed_speaker_transitions={
        chief_verifier: [doc_parse_caller],
        doc_parse_caller: [doc_parser],
        doc_parser: [education_cert_verifier]
    },
    speaker_transitions_type="allowed",
)

manager2 = autogen.GroupChatManager(groupchat=groupchat2, llm_config=llm_config)


In [10]:
# def parse_calling(recipient, messages, sender, config):
#     # return f"Can you provide the python function to call for parsing the documents?\n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}"
#     #print(f"{recipient.chat_messages_for_summary(sender)[-1]['content']}")
    
#     # if isinstance(recipient.chat_messages_for_summary(sender)[-1]['content'], dict):
#     #     return "TERMINATE"
#     # else:
#     return "I have some pdf documents in the directory '../data/', and I would like to parse the documents in this directory"


# def parsed_docs(recipient, messages, sender, config):
#     # return f"Can you provide the python function to call for parsing the documents?\n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}"
#     #print(f"{recipient.chat_messages_for_summary(sender)[-1]['content']}")
#     return "Here are the parsed documents to be classified"

def verify_doc(recipient, messages, sender, config):
    return f'''Verify the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

def verify_edu_doc(recipient, messages, sender, config):
    return f'''Verify the following content by comparing the documents in the directory path at '../templates/'. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''


chief_verifier.register_nested_chats(
    [
        {
            "recipient": rp_verifier,
            "message": verify_doc,
            "max_turns": 1,
            "summary_method": "reflection_with_llm",
            "summary_args": {"summary_prompt" : 
                "Return review into as JSON object only:"
                "{'Agent': 'Verifier Name', 'Verification': '', 'Remarks': ''}."
                "You should replace the value of Verifier Name with your role name, for example X Reviewer, where X represents the information you are reviewing." 
                "For the verification, provide a one sentence answer only, and provide the reason for your decision in the remarks key value pair"},
        },
        {
            "recipient": passport_verifier,
            "message": verify_doc,
            "max_turns": 1,
            "summary_method": "reflection_with_llm",
            "summary_args": {"summary_prompt" : 
                "Return review into as JSON object only:"
                "{'Agent': 'Verifier Name', 'Verification': '', 'Remarks': ''}."
                "You should replace the value of Verifier Name with your role name, for example X Reviewer, where X represents the information you are reviewing." 
                "For the verification, provide a one sentence answer only, and provide the reason for your decision in the remarks key value pair"},
        },
        {
            "recipient": manager2,
            "message": verify_edu_doc,
            "max_turns": 1,
            "summary_method": "reflection_with_llm",
            "summary_args": {"summary_prompt" : 
                "Return review into as JSON object only:"
                "{'Agent': 'Verifier Name', 'Verification': '', 'Remarks': ''}."
                "You should replace the value of Verifier Name with your role name, for example X Reviewer, where X represents the information you are reviewing." 
                "For the verification, provide a one sentence answer only, and provide the reason for your decision in the remarks key value pair"},
        },
        {
            "recipient": summary_verifier,
            "max_turns": 1,
            "message": "You are the summarizer for all verifications provided. Aggregrate the decisions by the verifications provided and provide a detailed remark for each of them"
        },
    ],
    trigger=manager,  # condition=my_condition,
)

In [12]:
result = user_proxy.initiate_chat(
    manager,
    # message="Parse the documents and classify each of them by a heading",
    message="I have some pdf documents in the directory '../data/', and I would like to know what type of documents they are (in terms of classification)",
)

User (to chat_manager):

I have some pdf documents in the directory '../data/', and I would like to know what type of documents they are (in terms of classification)

--------------------------------------------------------------------------------

Next speaker: Doc Parse Caller


>>>>>>>> USING AUTO REPLY...
Doc Parse Caller (to chat_manager):

***** Suggested tool call (call_acnc): parse_pdfs *****
Arguments: 
{"directory": "../data/"}
*******************************************************

--------------------------------------------------------------------------------

Next speaker: Doc Parser


>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING FUNCTION parse_pdfs...
Doc Parser (to chat_manager):

Doc Parser (to chat_manager):

***** Response from calling tool (call_acnc) *****
{"../data/ULB_registration_certificate.pdf": ["REGISTRATION CERTIFICATE FOR THE \nACADEMIC YEAR 2022-2023\nMme  GANEPOLA ACHCHIGE Koumudi Y.\nNo 47, Weliweriya west\n11710  Weliweriya\nSri Lanka\nBrussels, 0

/home/koumudi/Documents/Llama_Hack/LlamaHackers/venv/lib/python3.12/site-packages/autogen/oai/groq.py:279: UserWarning: Cost calculation not available for model llama-3.2-90b-text-preview
  warnings.warn(f"Cost calculation not available for model {model}", UserWarning)


Doc Parser (to chat_manager):

Doc Parser (to chat_manager):

***** Response from calling tool (call_qm23) *****
{"../templates/Template_enrollement.pdf": ["Annex 1 to the Ministerial Decree of 28 March 2022 establishing the standard forms \nreferred to in Articles 99, 103 and 104/3 of the Royal Decree of 8 October 1981 on access to the \nterritory, residence, establishment and\nremoval of aliens\nMODEL OF STANDARD FORM\nto obtain a visa or residence permit as a student (third-country national), as referred to in Article 99 \nof the Royal Decree of 8 October 1981 on access to the territory,\nthe residence, establishment and removal of aliens\nLogo of the higher education institution(optional):\nI, the undersigned(1).................................................. ..\nIn my capacity as representative of(2)……………………………………………………\nConfirm that the student below:\nName:\nFirst name:\nDate of birth:\nNationality:\n- Afinal registrationhas obtained the right to pursue full-time higher educat